In [1]:
import yaml
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
def load_credentials(credential, file = "./credentials.yml"):

    with open(file,"r") as c:
        credentials = yaml.safe_load(c)[credential]

    return credentials

In [3]:
credentials = load_credentials("blob_storage")

In [4]:
blob_service_client = BlobServiceClient\
    .from_connection_string(credentials['conn_string'])

In [10]:
# Create a unique name for the container
container_name = "raw-data"

try:
    container_client = blob_service_client.create_container(container_name)
    properties = container_client.get_container_properties()
except:
    print("Container already exists.")

Container already exists.


In [11]:
all_containers = blob_service_client.list_containers()
for c in all_containers:
    print(c.name)

raw-data
